In [231]:
#functions

import math
import random


# makes a ticket
def ticket_generator(t, o, a):
    ticket={
        'time': t,
        'owner': o,
        'action': a,
    }
    return ticket;


# depending on parameter (spread or cure), generates tau
def nextTime(rateParameter, n):
    taus=[0]*n
    for i in range(0,n): #start at zero, stops just short of n
        taus[i]=-math.log(1.0 - random.random()) / rateParameter
    return taus #generates n-length list, starting at index 0


#writes the edges to a network of n ppl where everyone is connected to everyone else 
def network_writer(n):
    connect=[0]*n
    for i in range(0, n): 
        a=list(range(n)) #generating a list that goes from 0 to n-1
        a.remove(i) #individual removes themselves from the network
        connect[i]=a #corresponds to the list of people person i is linked to
    return connect; #returns a list of lists with all the connections in the network

#ONCE NETWORKS ARE NOT SMALL WORLD, WILL NEED TO PASS IN CONNECT TO FUNCTION TO SEE WHO CAN BE INFECTED BY THE OWNER 
#(ow) AND TO GENERATE THE CORRECT LENGTH FOR Ts
#processes spread/infection tickets, returning a new timeline (with added tickets) and an updated list of states
def spreadtickethandler(tic, timeline, spreadparam, cureparam, ppl_state, t, t_max): #HEREEEEE
    ow=tic['owner']
    targ=tic['action'] #this action should not be -1 bc this is passed to spread ticket handler
    if (ow==targ):
        print('ow=targ, {:d}={:d}' .format(ow,targ))
#1
    if(ppl_state[ow]!=1):
        print('Error -- individual {:d} is not infected, but was passed to spreadtickethandler!' .format(ow))
        #break #is there a printf / how do you raise an error?  #SOMEHOW RAISE AND ERROR AND ESC
    else:
#2
        t=tic['time']
        if(ppl_state[targ]==1):
            print('{:d} is already infected! at time {:f}' .format(targ,t)) #targ=targ #just to make python shut up NOT REAL CODE
        else:
            ppl_state[targ]=1
            num_ppl=len(ppl_state) #number of people
            ts=[t]*(num_ppl-1) #list of t repeated num_ppl-1 times
            nT=nextTime(spreadparam, num_ppl-1) #generates taus
            cure_t=t+nextTime(cureparam, 1)[0] #using [0] in order to add an integer to t
            #curetimearray.append(cure_t) #curetime array -- purpose??
            spread_ts=[sum(x) for x in zip(ts,nT)] #using zip 
            acc=0
            for a in connect[ow]:
                if(spread_ts[acc]<cure_t and spread_ts[acc]<t_max): #HEREEE
                    tinf=ticket_generator(spread_ts[acc], targ, a)
                    add_to_timeline(tinf, timeline)
                acc=acc+1
            if cure_t<t_max: #HERE (unindent 2 lines below)
                tcure=ticket_generator(cure_t, targ, -1) #cure ticket
                add_to_timeline(tcure, timeline)
#3
        #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
#4
        spread_t2=t+nextTime(spreadparam,1)[0]
        cure_t_ow=t_max+1 #just so we don't get an error is cure_t_ow isn't generated in the while loop bc it's too big
        i=timeholder
        curefound=0 
        #looking for the curetime of each individual
        while curefound==0 and i<(len(timeline)-1): 
            tim=timeline[i]
            if(tim['owner']==ow): 
                if(tim['action']==-1):
                    cure_t_ow=tim['time']
                    curefound=1 #stop when cure is found
            i=i+1 
        if(spread_t2<cure_t_ow and spread_t2<t_max): #HEEREEE
            spread2tic=ticket_generator(spread_t2, ow, targ)
            add_to_timeline(spread2tic, timeline)
        return [timeline, ppl_state];
            
        
    
#processes cure tickets, returning the new states of ppl        
def curetickethandler(tic, ppl_state):
    ow=tic['owner']
    ppl_state[ow]=0
    #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
    return ppl_state

#adds new tickets to timeline
def add_to_timeline(tic, tim):
    if not tim:
        return [tic]
    ind=len(tim)-1
    end=0
    while (tic['time']<tim[ind]['time'] and end==0):
        ind=ind-1 
        if(ind<0):
            end=1
    ind=ind+1
    tim.insert(ind, tic)
    return tim

def new_ind(ind, ow):
    if(ind>ow):
        ind=ind-1
    return ind

In [232]:

#Test ticket generator

tic=ticket_generator(1,1,1)
timeline=[]
print(timeline)

timeline=add_to_timeline(tic,timeline)

tic1=ticket_generator(4,1,1)
timeline=add_to_timeline(tic1,timeline)
#print(timeline)

tic2=ticket_generator(5,1,1)
timeline=add_to_timeline(tic2,timeline)
#print(timeline)

tic3=ticket_generator(3.5,1,1)
timeline=add_to_timeline(tic3,timeline)
#print(timeline)

tic4=ticket_generator(7,1,1)
timeline=add_to_timeline(tic4,timeline)
print(timeline)



[]
[{'time': 1, 'owner': 1, 'action': 1}, {'time': 3.5, 'owner': 1, 'action': 1}, {'time': 4, 'owner': 1, 'action': 1}, {'time': 5, 'owner': 1, 'action': 1}, {'time': 7, 'owner': 1, 'action': 1}]


In [233]:
#main

del timeline #deletes timeline from previous run

sick_ind=5
spreadparam=.5
cureparam=.3
curetimearray=[];
timeline=[]
num_ppl=100
ppl_state=[0]*num_ppl 
ppl_state[sick_ind]=1 #make a record of everyone's current state, starting with 1 infected person
connect=network_writer(num_ppl) #make a network of connections btwn the ppl -- everyone is connected to everyone rn
tinfinit=nextTime(spreadparam, num_ppl-1) #generate the time of the first infection THIS IS NOW ALL THE TIMES -- one for each
tcureinit=nextTime(cureparam, 1)[0] #generate the time of sick_ind's cure
while(all(i < tcureinit for i in tinfinit)): #for the very rare case that no individuals get infected before individual 0 gets cured
    tcureinit=nextTime(cureparam,1) #ensure that that time occurs after patient 0 has infected someone -- just so our sim goes somewhere
smallinds=[] #isolate only those times that are smaller than cure time
acc=0
for i in connect[sick_ind]:        
    if tinfinit[acc]<tcureinit:
        smallinds.append(i)
    acc=acc+1    
timeline=[]#initialize the timeline before any tickets are added
for i in range (len(smallinds)):
    init_inf=ticket_generator(tinfinit[new_ind(smallinds[i], sick_ind)], sick_ind, smallinds[i])#generate a ticket for the first infection 
    timeline=add_to_timeline(init_inf, timeline) #put these 2 tickets on timeline
#print(timeline)
init_cure=ticket_generator(tcureinit, sick_ind, -1) # generate a ticket for the first cure
timeline=add_to_timeline(init_cure, timeline)
#print(timeline)
timeholder=0 #timeholder is the index where we are on the timeline (start at 0)
t=0 #we start at a time of 0 
t_max=5 #we want to go to 500 time units



In [234]:
#ok time for the while loop
while t<t_max:
    print(t)
    if timeholder>(len(timeline)-1):
        print('end')
        break
    nexttic=timeline[timeholder] #look at the ticket corresponding to where we are on the timeline
    if (nexttic['action']==-1): #if it's a cure ticket, use the corresponding handler
        ppl_state=curetickethandler(nexttic, ppl_state)
    else: #if it's a spread ticket, use the corresponding handler
        vec=spreadtickethandler(nexttic, timeline, spreadparam, cureparam, ppl_state, t, t_max) #HEEREEE
        timeline=vec[0]
        ppl_state=vec[1]
    t=nexttic['time'] #new time based on the ticket we just read

    
    timeholder=timeholder+1 #increment timeholder
    


0
0.022562570979855007
0.04931545885220999
0.05356885856793525
0.060905404975542136
0.06239048548798265
0.06624220348787253
0.0706502302635923
0.07158217669503467
0.07576688784470784
1 is already infected! at time 0.075842
0.07584223850256308
0.07672383426662288
0.07780402125940797
0.07945361874252324
0.0801979923423695
0.08051259307006757
0.08084896348490087
0.08134077833757881
0.08187375693575986
0.08226656710998644
0.08312147050939339
0.08400482630563556
0.08431578945999714
63 is already infected! at time 0.084723
0.08472340259395199
0.0878776849464811
0.08886252492680533
1 is already infected! at time 0.088895
0.08889504733211392
0.0898648292521885
1 is already infected! at time 0.090526
0.09052563772393207
0.09081094432473728
0.09163373532733277
59 is already infected! at time 0.092303
0.09230268204822904
0.09252088426511151
0.09262920473093894
0.09275561670341641
0.09349825038514123
38 is already infected! at time 0.094410
0.09440994825450419
86 is already infected! at time 0.094

0.15703690546846852
42 is already infected! at time 0.157112
0.15711195023043095
29 is already infected! at time 0.157126
0.1571257683861951
86 is already infected! at time 0.157155
0.15715530058632715
29 is already infected! at time 0.157678
0.1576777412806824
65 is already infected! at time 0.157790
0.15778969753265754
30 is already infected! at time 0.157825
0.15782514952115953
4 is already infected! at time 0.158206
0.15820578126922413
50 is already infected! at time 0.158208
0.15820750212110113
66 is already infected! at time 0.158641
0.15864135864895665
33 is already infected! at time 0.158890
0.15889032491384456
28 is already infected! at time 0.158935
0.15893465021982664
51 is already infected! at time 0.158993
0.15899280165736157
69 is already infected! at time 0.159119
0.15911893792022958
56 is already infected! at time 0.159528
0.1595278471625837
0.1596760363386133
24 is already infected! at time 0.159957
0.15995717131352244
27 is already infected! at time 0.160004
0.1600039

0.19198673875669936
29 is already infected! at time 0.192001
0.1920011776108015
21 is already infected! at time 0.192148
0.1921477475104905
60 is already infected! at time 0.192201
0.19220059105886533
63 is already infected! at time 0.192247
0.1922472817166498
66 is already infected! at time 0.192410
0.19240953768414107
41 is already infected! at time 0.192873
0.192873477452708
96 is already infected! at time 0.193091
0.19309073665034565
67 is already infected! at time 0.193713
0.19371288451408142
2 is already infected! at time 0.193733
0.19373316307374372
30 is already infected! at time 0.194090
0.19408964456069472
15 is already infected! at time 0.194159
0.1941587978894891
72 is already infected! at time 0.194266
0.194265605756869
30 is already infected! at time 0.194304
0.19430384921158855
28 is already infected! at time 0.194323
0.19432309940605735
20 is already infected! at time 0.194779
0.19477893987965553
66 is already infected! at time 0.194948
0.194948284427549
87 is already i

31 is already infected! at time 0.220399
0.22039928126916464
85 is already infected! at time 0.220936
0.22093555271692064
10 is already infected! at time 0.221229
0.22122883226853834
25 is already infected! at time 0.221315
0.22131548992816208
26 is already infected! at time 0.221564
0.22156352505427623
39 is already infected! at time 0.221595
0.2215953405916051
6 is already infected! at time 0.221645
0.22164473102707583
99 is already infected! at time 0.221793
0.221792735081627
32 is already infected! at time 0.221845
0.2218447762525301
67 is already infected! at time 0.223097
0.22309655543266027
18 is already infected! at time 0.223732
0.22373199817383083
63 is already infected! at time 0.223788
0.22378820987862186
64 is already infected! at time 0.223928
0.2239279602959368
7 is already infected! at time 0.223962
0.2239618248793609
98 is already infected! at time 0.224036
0.22403564960656538
58 is already infected! at time 0.224493
0.22449267436439574
1 is already infected! at time 0

0.2505257552279761
19 is already infected! at time 0.250666
0.2506655228420842
34 is already infected! at time 0.251177
0.25117699743866817
17 is already infected! at time 0.251353
0.2513529113798467
76 is already infected! at time 0.251386
0.2513856968373005
41 is already infected! at time 0.251449
0.2514486725896704
27 is already infected! at time 0.251965
0.2519652882257306
16 is already infected! at time 0.252691
0.2526906636107422
8 is already infected! at time 0.252898
0.25289769787891075
57 is already infected! at time 0.252919
0.25291919408724073
11 is already infected! at time 0.252938
0.2529380786871517
11 is already infected! at time 0.253299
0.2532989877348225
16 is already infected! at time 0.253335
0.2533352073652816
26 is already infected! at time 0.253816
0.2538155583693716
84 is already infected! at time 0.254105
0.254104907026233
23 is already infected! at time 0.254221
0.25422121095245687
0.2549449754216222
25 is already infected! at time 0.255197
0.25519721069801304

...

4.737888352181148
83 is already infected! at time 4.737923
4.7379231602735885
94 is already infected! at time 4.737994
4.7379944420267295
83 is already infected! at time 4.738083
4.738082910295953
49 is already infected! at time 4.738515
4.738514872339234
42 is already infected! at time 4.738572
4.738571917141611
43 is already infected! at time 4.738576
4.738575705524449
50 is already infected! at time 4.739194
4.739193987324602
23 is already infected! at time 4.739208
4.739208283440227
67 is already infected! at time 4.739288
4.739287790754358
99 is already infected! at time 4.739588
4.73958829870908
64 is already infected! at time 4.739609
4.739608986144128
97 is already infected! at time 4.739815
4.739815378338228
5 is already infected! at time 4.740309
4.7403090607974265
6 is already infected! at time 4.740473
4.740472616728211
88 is already infected! at time 4.740520
4.740519933428654
5 is already infected! at time 4.740844
4.740843532492407
38 is already infected! at time 4.74086

4.768034481937169
11 is already infected! at time 4.768094
4.768093661130003
74 is already infected! at time 4.768327
4.7683265119551965
16 is already infected! at time 4.768416
4.768416222374558
86 is already infected! at time 4.769383
4.769383322939174
75 is already infected! at time 4.769439
4.769438774153913
71 is already infected! at time 4.769488
4.769487562441004
42 is already infected! at time 4.769542
4.769542436479735
15 is already infected! at time 4.770695
4.770694920253428
45 is already infected! at time 4.771062
4.771062285456968
5 is already infected! at time 4.771358
4.771357899716488
18 is already infected! at time 4.771544
4.771544341942311
86 is already infected! at time 4.771929
4.771928702618544
33 is already infected! at time 4.772225
4.772224954962096
34 is already infected! at time 4.772682
4.772682390551396
62 is already infected! at time 4.772840
4.772840483365177
76 is already infected! at time 4.772988
4.772988390532815
19 is already infected! at time 4.7730

4.822884256429498
11 is already infected! at time 4.822944
4.8229442629264225
69 is already infected! at time 4.823214
4.823213586294589
33 is already infected! at time 4.823254
4.823254473964892
66 is already infected! at time 4.823257
4.823256909203446
41 is already infected! at time 4.823631
4.823630694573659
91 is already infected! at time 4.823688
4.823688269415311
5 is already infected! at time 4.823947
4.823946685288868
90 is already infected! at time 4.824113
4.824113244554004
44 is already infected! at time 4.824155
4.824154809780633
89 is already infected! at time 4.824190
4.824190310693276
92 is already infected! at time 4.824211
4.824211245874616
41 is already infected! at time 4.824215
4.824214826143904
57 is already infected! at time 4.824814
4.824814347700144
59 is already infected! at time 4.825078
4.825078460301103
67 is already infected! at time 4.825112
4.825111768872127
28 is already infected! at time 4.825115
4.825115406590804
75 is already infected! at time 4.8251

4.882936179903996
41 is already infected! at time 4.882953
4.882952580787679
58 is already infected! at time 4.883230
4.883229805998101
46 is already infected! at time 4.883269
4.883269100241504
80 is already infected! at time 4.883413
4.883413007309512
53 is already infected! at time 4.883498
4.883497657324318
16 is already infected! at time 4.883551
4.883550980039452
37 is already infected! at time 4.883617
4.8836166221366115
19 is already infected! at time 4.883710
4.883710211087719
54 is already infected! at time 4.883812
4.883811791102823
94 is already infected! at time 4.883915
4.883914892153081
73 is already infected! at time 4.884081
4.8840809711403725
89 is already infected! at time 4.884167
4.884167372412238
41 is already infected! at time 4.884216
4.884215885008614
23 is already infected! at time 4.884294
4.884294077002315
15 is already infected! at time 4.884338
4.884338048898109
7 is already infected! at time 4.884418
4.884417776417626
63 is already infected! at time 4.884

96 is already infected! at time 4.968118
4.968118295598233
33 is already infected! at time 4.968225
4.968224582277135
63 is already infected! at time 4.968543
4.968543291159222
45 is already infected! at time 4.968738
4.968737632005261
33 is already infected! at time 4.968975
4.968974564131449
20 is already infected! at time 4.969100
4.9691002274946205
66 is already infected! at time 4.969106
4.969106045334949
39 is already infected! at time 4.969220
4.969220040851129
53 is already infected! at time 4.969551
4.96955122527841
34 is already infected! at time 4.969603
4.969603076622067
10 is already infected! at time 4.969632
4.969631850303271
26 is already infected! at time 4.969999
4.969998956573656
95 is already infected! at time 4.970075
4.970074613963902
10 is already infected! at time 4.970134
4.970134117751935
38 is already infected! at time 4.970252
4.970252119073644
40 is already infected! at time 4.970538
4.970538119873565
55 is already infected! at time 4.970804
4.9708040816403

In [235]:


print(timeline)

40
{'time': 0.09832442883023707, 'owner': 30, 'action': 20}
[{'time': 0.022562570979855007, 'owner': 5, 'action': 70}, {'time': 0.04931545885220999, 'owner': 70, 'action': 16}, {'time': 0.05356885856793525, 'owner': 70, 'action': 14}, {'time': 0.060905404975542136, 'owner': 70, 'action': 57}, {'time': 0.06239048548798265, 'owner': 16, 'action': 73}, {'time': 0.06624220348787253, 'owner': 73, 'action': 30}, {'time': 0.0706502302635923, 'owner': 70, 'action': 88}, {'time': 0.07158217669503467, 'owner': 14, 'action': 1}, {'time': 0.07576688784470784, 'owner': 16, 'action': 10}, {'time': 0.07584223850256308, 'owner': 73, 'action': 1}, {'time': 0.07672383426662288, 'owner': 70, 'action': 86}, {'time': 0.07780402125940797, 'owner': 10, 'action': 46}, {'time': 0.07945361874252324, 'owner': 16, 'action': 63}, {'time': 0.0801979923423695, 'owner': 46, 'action': 31}, {'time': 0.08051259307006757, 'owner': 31, 'action': 50}, {'time': 0.08084896348490087, 'owner': 57, 'action': 60}, {'time': 0.081

In [ ]:
#test 

num_ppl=5
tinfinit=[1, 3, 2, 4, 5]
tcureinit=3

smallinds=[] #ERROR -- ULTIMATE GOAL is to isolate only those times that are smaller than cure time
for i in range (num_ppl-1):
    if tinfinit[i]<tcureinit:
        smallinds.append(i)

print(smallinds) 


In [22]:
spreadparam=.1
cureparam=.05
curetimearray=[];
timeline=[]
num_ppl=100
ppl_state=[0]*num_ppl 
ppl_state[0]=1 #make a record of everyone's current state, starting with 1 infected person
connect=network_writer(num_ppl) #make a network of connections btwn the ppl -- everyone is connected to everyone rn

tinfinit=nextTime(spreadparam, num_ppl-1) #generate the time of the first infection THIS IS NOW ALL THE TIMES -- one for each
tcureinit=nextTime(cureparam, 1)[0] #generate the time of patient 0's cure
while(sum(tcureinit>tinfinit)==0):
    tcureinit=nextTime(cureparam) #ensure that that time occurs after patient 0 has infected someone -- just so our sim goes somewhere
tinfinit.sort()   
smallinds=tinfinit<tcureinit
tinfinit=tinfinit[smallinds.index(1)]

TypeError: '>' not supported between instances of 'int' and 'list'

In [21]:
print(all(i < tcureinit[0] for i in tinfinit))

False


In [19]:
l1=[30, 34, 56]

all(i >= 30 for i in l1)

True

In [20]:
print(tcureinit[0])
print(tinfinit)

[0]
[18.900137608539172, 2.7595800380249704, 0.10891789736790652, 17.368996656500048, 2.45294843694846, 6.704430433308595, 0.1748955419401317, 4.8646342241154334, 11.385189763283636, 9.786465635853672, 0.11865745866570593, 6.04017066517331, 0.13194557214631525, 6.156042352291014, 32.92858509084251, 15.94384666225129, 6.194619658550465, 22.543115922067418, 3.4500118507255255, 6.655992288418583, 4.856863844482613, 8.595885931909567, 4.979684889996908, 0.646446599445525, 29.43224445115833, 12.046640268366074, 1.8582530884557515, 7.591699611724711, 11.030713762950151, 7.864876036411042, 12.724499345639916, 41.24963651337464, 18.183572075994398, 17.43939464120913, 1.9097157689220872, 15.819034496017071, 0.9813997084787515, 19.343086961366332, 7.619708482723527, 27.088677683866933, 9.386626724158903, 7.645786855686298, 15.308159662774958, 7.118874520953093, 7.754283793560688, 17.492311345666334, 13.301132055967459, 13.763362582429172, 1.2545543857978763, 21.75418121746059, 8.899197436873587,

In [ ]:
#old spreadtickethandler, give some ow=targ errors
def spreadtickethandler(tic, timeline, spreadparam, cureparam, ppl_state, t, t_max): #HEREEEEE
    ow=tic['owner']
    targ=tic['action'] #this action should not be -1 bc this is passed to spread ticket handler
    if (ow==targ):
        print('ow=targ, {:d}={:d}' .format(ow,targ))
#1
    if(ppl_state[ow]!=1):
        print('Error -- individual {:d} is not infected, but was passed to spreadtickethandler!' .format(ow))
        #break #is there a printf / how do you raise an error?  #SOMEHOW RAISE AND ERROR AND ESC
    else:
#2
        t=tic['time']
        if(ppl_state[targ]==1):
            print('{:d} is already infected! at time {:f}' .format(targ,t)) #targ=targ #just to make python shut up NOT REAL CODE
        else:
            ppl_state[targ]=1
            num_ppl=len(ppl_state) #number of people
            ts=[t]*(num_ppl-1) #list of t repeated num_ppl-1 times
            nT=nextTime(spreadparam, num_ppl-1) #generates taus
            cure_t=t+nextTime(cureparam, 1)[0] #using [0] in order to add an integer to t
            #curetimearray.append(cure_t) #curetime array -- purpose??
            spread_ts=[sum(x) for x in zip(ts,nT)] #using zip 
            acc=0
            for a in connect[ow]:
                if(spread_ts[acc]<cure_t and spread_ts[acc]<t_max): #HEREEE
                    tinf=ticket_generator(spread_ts[acc], targ, a)
                    add_to_timeline(tinf, timeline)
                acc=acc+1
            if cure_t<t_max: #HERE (unindent 2 lines below)
                tcure=ticket_generator(cure_t, targ, -1) #cure ticket
                add_to_timeline(tcure, timeline)
#3
        #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
#4
        spread_t2=t+nextTime(spreadparam,1)[0]
        cure_t_ow=t_max+1 #just so we don't get an error is cure_t_ow isn't generated in the while loop bc it's too big
        i=timeholder
        curefound=0 
        #looking for the curetime of each individual
        while curefound==0 and i<(len(timeline)-1): 
            tim=timeline[i]
            if(tim['owner']==ow): 
                if(tim['action']==-1):
                    cure_t_ow=tim['time']
                    curefound=1 #stop when cure is found
            i=i+1 
        if(spread_t2<cure_t_ow and spread_t2<t_max): #HEEREEE
            spread2tic=ticket_generator(spread_t2, ow, targ)
            add_to_timeline(spread2tic, timeline)
        return [timeline, ppl_state];

In [ ]:
            smallinds=[] #isolate only those times that are smaller than cure time
            acc=0
            for i in connect[ow]:        
                if ts[acc]<cure_t:
                    smallinds.append(i)
                acc=acc+1    
            for i in range (len(smallinds)):
                tinf=ticket_generator(ts[new_ind(smallinds[i], ow)], ow, smallinds[i])#generate a ticket for the first infection 
                timeline=add_to_timeline(tinf, timeline) #put these 2 tickets on timeline                 
                
            if cure_t<t_max: #HERE (unindent 2 lines below)
                tcure=ticket_generator(cure_t, targ, -1) #cure ticket
                add_to_timeline(tcure, timeline)